In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

checkpoint1

In [3]:
dict={}
i=1
for col in mnist["data"].columns:
    dict[col]=f"X{i}"
    i+=1
Data=mnist["data"]
Data.rename(columns=dict,
          inplace=True)

c:\users\rohit\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [4]:
for i in range(0,10):
    Data.insert(len(Data.columns),f"Y{i}",0,True)
Target=mnist["target"]
for i in range(0,7000):
    t=int(Target[i])
    Data.iloc[i,-10+t]=1

c:\users\rohit\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [5]:
Data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
Training_Set=Data.iloc[0:5000,:]
Testing_Set=Data.iloc[5000:7000,:]

Check Point 2
Implementing forward propagations to a row of Data,
inputs:X1-X784 list , W(1),w(2)
outputs:activations of hidden layer and output layer

Creatinng W1 AND W2 of dimensions n X 785 and 10 x n+1


In [7]:
n=78

In [31]:
e1=math.sqrt(6)/(math.sqrt(784)+math.sqrt(n))
W1=2*e1*(np.random.rand(n,785)-1/2) 
#uniform distribution from(-e1,e1)

In [34]:
e2=math.sqrt(6)/(math.sqrt(n)+math.sqrt(10))
W2=2*e2*(np.random.rand(10,n+1)-1/2)

In [10]:
def g(x):
    if(x>500):
        return 1
    if(x<-500):
        return 0
    return 1/(1+math.exp(-x))

Sigmoid Function

Check Point 3

In [11]:
def forward_propagation(W1,W2,data_point):
    x=np.array(data_point[0:784])
    y=np.array(data_point[784:])
    z1=np.dot(W1[:,1:],x)+W1[:,0]
    a1= np.array([g(xi) for xi in z1])
    z2=np.dot(W2[:,1:],a1)+W2[:,0]
    a2=np.array([g(xi) for xi in z2])
    return x,a1,a2,y
    

forward_propagation(W1,W2,Data.iloc[0,:])

checkpoint 4
creating cross entropy cost function


In [12]:
def Point_Cost(a2,y):
    Cost=0
    for i in range(0,10):
        Cost=Cost+y[i]*math.log(a2[i])+(1-y[i])*math.log(1-a2[i])
    return Cost
    

checkpoint 5
Applying backpropagation to get delta matrix and individual deltas


In [51]:
def backward_propagation(x,a1,a2,y,W1,W2):
    del_2=a2-y
    del_g=np.array([a*(1-a)for a in a1])
    partial_del_1=np.dot(del_2,W2[:,1:])
    del_1=np.array([partial_del_1[i]*a1[i]*(1-a1[i]) for i in range(0,a1.size)])
    return del_1,del_2

In [111]:
def point_gradient(X,W1,W2,a1,del_1,del_2):
    W1=0*W1
    W2=0*W2
    for i in range(0,W1.shape[0]):
        W1[i,0]=del_1[i]
        for j in range(1,W1.shape[1]):
            W1[i,j]=del_1[i]*X[j-1]
    for i in range(0,W2.shape[0]):
        W2[i,0]=del_2[i]
        for j in range(1,W2.shape[1]):
            W2[i,j]=del_2[i]*a1[j-1]
    return W1,W2

In [112]:
x,a1,a2,y=forward_propagation(W1,W2,Data.iloc[0,:])

In [113]:
del_1,del_2=backward_propagation(x,a1,a2,y,W1,W2)

In [114]:
del_W1,del_W2=point_gradient(Data.iloc[0,0:784],W1,W2,a1,del_1,del_2)

checkpoint 7 creating gradient descent


checkpoint8 imlementing the above using gradient descent 


creating prediction function


In [59]:
def predict(X,W1,W2):
    z1=np.dot(W1[:,1:],x)+W1[:,0]
    a1= np.array([g(xi) for xi in z1])
    z2=np.dot(W2[:,1:],a1)+W2[:,0]
    a2=np.array([g(xi) for xi in z2])
    result=np.where(a2==np.amax(a2))[0]
    return result

checkpoint 9 implementing batch gradient descent